Para correr este notebook é necessária a instalação do Python e de um conjunto de bibliotecas. Para facilitar a instalação, recomenda-se a instalação do [Anaconda](https://www.anaconda.com/products/individual) e criação de um ambiente a partir do ficheiro geopreprocess.yml disponibilizado na pasta [Conda and PIP env]. Alternativamente, pode-se instalar as bibliotecas usando o comando pip install -r requirements_geopreprocess.txt na linha de comandos

# DESCRIÇÃO
Este notebook é dedicado à geocodificação (identificação da coordenada x,y) dos registos da base de dados PrimeYield com informação em falta relativamente à localização geográfica.
A georeferenciação é realizada com base na variável código postal e recorrendo à API do Google Maps.


Configuração do projeto

In [ ]:
# pip install geopy

In [ ]:
import pandas as pd 
import numpy as np

import geopandas as gpd


import seaborn as sns

# Livrarias para ler dados do openstreetmaps e representação da rede de estradas como um grafo
import osmnx as ox
import networkx as nx

import re

#import pgeocode as pgc
#import geopy

import time

import matplotlib.pyplot as plt

import os

import pickle 



In [ ]:
from tqdm import tqdm
#{progress in percentage} | {bar} | {processed}/{total} [{elapsed time}<{remaining time}, {number of iterations per seconds} it/s]

In [ ]:
os.getcwd()

# Directorias

## DADOS base de dados PrimeYield

In [ ]:

open_file = open('../Data/Inputs/db_primeyield_ini.piclo', "rb")
records_py_nogps = pickle.load(open_file)
open_file.close()

In [ ]:
records_py_nogps["geocod_query"] = records_py_nogps["ADD_POSTAL_CODE"].astype(str) + ", Aveiro, Portugal"
records_py_nogps

In [ ]:
from geopy.geocoders import GoogleV3
gkey = 'INSERT YOUR KEY HERE'
geolocator = GoogleV3(api_key=gkey)

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
records_py_nogps['location'] = records_py_nogps['geocod_query'].apply(geocode)


In [ ]:
records_py_nogps['location']

In [ ]:
from shapely.geometry import Point
records_py_nogps['point'] = records_py_nogps['location'].apply(lambda loc: tuple(loc.point[0:2]) if loc else None)
records_py_nogps['point'] = records_py_nogps['point'].apply(Point)

In [ ]:
records_py_nogps['point'] 

In [ ]:
records_py_geocoded = records_py_nogps

In [ ]:
# Save to pikle file
objects_store = records_py_geocoded
open_file = open('../Data/Outputs/db_primeyield_ini.piclo', "wb")
pickle.dump(objects_store, open_file)
open_file.close()


""" open_file = open('../Data/Outputs/db_primeyield_ini.piclo', "rb")
records_py_nogps = pickle.load(open_file)
open_file.close() """